In [1]:
import polars as pl
from pathlib import Path

## Data

In [2]:
locations = pl.scan_parquet(Path("data/cleaned/locations.parquet"))
jobs = pl.scan_parquet(Path("data/cleaned/jobs.parquet"))
companies = pl.scan_parquet(Path("data/cleaned/companies.parquet"))


## 1. What location has the most jobs that are either posted or expired?

In [3]:
(
    locations
    .join(jobs, on="zip_code")
    .filter(pl.col("state").is_in(["posted", "expired"]))
    .group_by("location")
    .agg(jobs=pl.count("id"))
    .sort("jobs", descending=True)
    .collect()
    .to_dicts()
)[0]

{'location': 'Tarth', 'jobs': 15}

## 2. What month had the most cancelled jobs?

In [4]:
(
    jobs
    .filter(pl.col("state").eq("cancelled"))
    .group_by("year", "month")
    .agg(jobs=pl.count("id"))
    .sort("jobs", descending=True)
    .collect()
    .to_dicts()
)[0]

{'year': 2021, 'month': 6, 'jobs': 5}

## 3. Which company has the highest ratio of posted jobs to employee count?

There's no job data available for Jumpstart Corp. 

In [5]:
(
    companies
    .join(jobs, on="company_id")
    .filter(pl.col("state").eq("posted"))
    .group_by("company_name", "number_of_employees")
    .agg(posted_jobs=pl.count("id"))
    .with_columns(posted_jobs_per_employee=pl.col("posted_jobs") / pl.col("number_of_employees"))
    .sort("posted_jobs_per_employee", descending=True)
    .collect()
    .to_dicts()
)[0]

{'company_name': 'Best Corp.',
 'number_of_employees': 229,
 'posted_jobs': 4,
 'posted_jobs_per_employee': 0.017467248908296942}

### 4. Develop the SQL to define a dimensional model schema for this data. Document (or be ready to discuss) any design decisions that you make.

1. Company Dimension

In [6]:
CREATE TABLE dim_company
(
    company_id INT PRIMARY KEY,
    company_name VARCHAR(255),
    establishment_date DATE,
    number_of_employees INT
);

SyntaxError: invalid syntax (1704445078.py, line 1)

2. Location Dimension

In [ ]:
CREATE TABLE dim_location
(
    zip_code INT PRIMARY KEY,
    location VARCHAR(255)
);

3. Fact Table

In [ ]:
CREATE TABLE jobs
(
    id INT PRIMARY KEY,
    state VARCHAR(255),
    price FLOAT,
    company_id INT,
    date DATE,
    month SMALLINT,
    year INT,
    zip_code INT,
    FOREIGN KEY (company_id) REFERENCES dim_company(company_id),
    FOREIGN KEY (zip_code) REFERENCES dim_location(zip_code)
);

#### Design Considerations

- Primary Keys: Each table has a primary key (company_id, zip_code, and id for the fact table), which uniquely identifies a row within those tables.

- Foreign Keys in Fact Table: The fact_transaction table includes foreign keys to dim_company and dim_location. This enforces referential integrity between the transactions and companies/locations involved. It allows the fact table to remain lean and optimized for queries that join on these dimensions for descriptive analysis.

- Normalization: The schema shows a normalized form to avoid redundancy and ensure data integrity. This is particularly important for the company and location information, which might relate to many transactions.

- Granularity: The fact table's granularity is at the individual transaction level, containing foreign keys back to the dimension tables for descriptive attributes.

- Month and Year Columns: Including month and year as separate columns in the fact_transaction table can speed up queries that aggregate data by these time units without requiring extraction from the date column.

#### Additional Considerations (Not Implemented):

- Job Description: The job description is not included in the schema. If this information is needed for analysis, it could be stored in a separate table linked to the fact table by a foreign key.

- Keys and Indexes: The schema does not include keys or indexes for performance optimization. Depending on the database system and query patterns, these could be added to improve query performance.

### 5. Using your dimensional model, write a SQL query that returns a list of jobs for each company, ordered and enumerated within each group by the posted_at date

In [ ]:
WITH OrderedJobs AS (
  SELECT
    j.id AS job_id,
    c.company_name,
    j.date AS posted_date,
    ROW_NUMBER() OVER(PARTITION BY j.company_id ORDER BY j.date) AS job_sequence_number
  FROM jobs j
  JOIN dim_company c ON j.company_id = c.company_id
)

SELECT *
FROM OrderedJobs
ORDER BY company_name, posted_date;

### 6. Discuss how you would obtain and model information (within your schema) about the duration of jobs (from posted to expired states).

The schema currently contains `date` which is referring to the date at which the job was posted. To model the duration of jobs, we could add a new column to the fact table called `expired_at`. When a new job listing is captured, both the start (posting) and end (expiration) points should be recorded.

Using the newly added `expired_at` field, we can calculate the difference between the `posted_at` and `expired_at` dates. This would allow us to analyze the length of time jobs are active and identify trends or patterns in job duration.

### 7. Provide a visualization showing the trend of the number of active jobs over time. Also include the company name data in this visualization.

Assuming active jobs implies jobs that are currently posted and not expired or cancelled.

In [6]:
import plotly.graph_objects as go
from datetime import date

In [7]:
def create_area_chart(x: list[date], y: list[list[float]], name: list[str]) -> go.Figure:
    assert len(y) == len(name)

    fig = go.Figure()
    for i in range(len(y)):
        fig.add_trace(go.Scatter(x=x, y=y[i], mode="lines", name=name[i]))

    fig.update_layout(
        title="Cumulative Jobs Posted Over Time",
        xaxis_title="Date",
        yaxis_title="Number of Jobs Posted",
        legend_title="Companies"
    )

    return fig

In [8]:
df = (
    companies
    .join(jobs, on="company_id")
    .filter(pl.col("state").eq("posted"))
    .group_by("company_name", "date")
    .agg(posted_jobs=pl.count("id"))
    .collect()
)

In [9]:
dates = df.select("date").unique()
company_names = df.select("company_name").unique()["company_name"].to_list()

cum_df = (
    dates
    .join(df.select("company_name").unique(), on="date", how="cross")
    .join(df, on=["company_name", "date"], how="left")
    .with_columns(posted_jobs=pl.col("posted_jobs").fill_null(0))
    .sort("date")
    .with_columns(cum_posted_jobs=pl.col("posted_jobs").cum_sum().over("company_name"))
)

In [10]:
fig = create_area_chart(
    x=dates.sort("date")["date"].to_list(),
    y=[cum_df.filter(pl.col("company_name").eq(company_name)).select("cum_posted_jobs")["cum_posted_jobs"].to_list() for company_name in company_names],
    name=company_names
)

In [11]:
fig.show()

Average Price of Posted Jobs by Companies

In [14]:
(
    companies
    .join(jobs, on="company_id")
    .group_by("company_name", "number_of_employees", "establishment_date")
    .agg(avg_price=pl.mean("price"), min_price=pl.min("price"), max_price=pl.max("price"), std_price=pl.std("price"))
    .sort("avg_price", descending=True)
    .collect()
)

company_name,number_of_employees,establishment_date,avg_price,min_price,max_price,std_price
str,i64,date,f64,f64,f64,f64
"""Infinite Solut…",436,2021-03-12,750.949091,46.15,5898.0,1711.470019
"""Delta Inc.""",662,2020-11-06,317.613333,63.72,488.1,143.258997
"""Global Enterpr…",896,2022-02-03,294.446875,1.77,466.63,174.347309
"""Acme Inc.""",404,2019-12-31,281.751111,36.27,464.94,157.763981
"""High Hopes Inc…",812,2020-03-17,280.988,12.39,449.49,167.136731
"""Echo Enterpris…",255,2022-05-05,276.063333,4.83,499.23,194.996285
"""Fast Track Inc…",374,2020-03-08,257.565,67.15,440.43,130.624846
"""Bright Future …",222,2020-11-14,239.06125,77.28,498.05,150.577506
"""Best Corp.""",229,2021-03-21,212.288,11.83,440.71,174.053786


In [13]:
(
    companies
    .join(jobs, on="company_id")
    .filter(pl.col("state").eq("posted"))
    .group_by("company_name", "number_of_employees")
    .agg(posted_jobs=pl.count("id"))
    .with_columns(posted_jobs_per_employee=pl.col("posted_jobs") / pl.col("number_of_employees"))
    .sort("posted_jobs_per_employee", descending=True)
    .collect()
)

company_name,number_of_employees,posted_jobs,posted_jobs_per_employee
str,i64,u32,f64
"""Best Corp.""",229,4,0.017467
"""Bright Future …",222,3,0.013514
"""Echo Enterpris…",255,3,0.011765
"""Delta Inc.""",662,4,0.006042
"""Global Enterpr…",896,5,0.00558
"""Fast Track Inc…",374,2,0.005348
"""Infinite Solut…",436,2,0.004587
"""High Hopes Inc…",812,1,0.001232
